In [ ]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import tiktoken 
tokenizer = tiktoken.get_encoding("gpt2")
with open('gutenbergtext.txt', 'r') as f:
    _tokens = tokenizer.encode_ordinary(f.read())

# sample tokens
print("\nSamples from tokenization: ")
print([tokenizer.decode_single_token_bytes(token) for token in _tokens[150:170]])


Samples from tokenization: 
[b' the', b' waters', b' which', b' were', b'\n', b'under', b' the', b' firm', b'ament', b' from', b' the', b' waters', b' which', b' were', b' above', b' the', b'\n', b'f', b'irm', b'ame']


In [ ]:
num_tokens = len(_tokens)
vocab = list(set(_tokens))
vocab_size = len(vocab)
# ordinal_encodings

otoe = {i : vocab[i] for i in range(vocab_size)}
etoo = {vocab[i] : i for i in range(vocab_size)}
# otoe = {i : _tokens[i] for i in range(num_tokens)}
# etoo = {_tokens[i] : i for i in range(num_tokens)}
ordinalize = lambda t : etoo[t]
deordinalize = lambda t : otoe[t]

tokens = [ordinalize(t) for t in _tokens]
assert(_tokens == [deordinalize(t) for t in tokens])
print(f'number of tokens = {len(tokens)}')
assert(max(tokens) == vocab_size - 1)

number of tokens = 52708


In [ ]:

import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
batch_size = 8
block_size = 32
max_iters = 10000
eval_iters = 500
eval_interval = 500
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
n_embd = 32
n_head = 8
n_layer = 8
print("device:" + device)
dropout = 0.2

device:cuda


In [ ]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out
    








class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    






class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    





class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
    



class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        # if loss is not None:
        #     if loss < 5.5:  
        #         learning_rate = 1e-3

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [ ]:

data = torch.tensor(tokens, dtype=torch.long,device=device)
print(data.shape, data.dtype)

torch.Size([52708]) torch.int64


In [ ]:

train_data = data[:int(num_tokens * 0.9)]
val_data = data[int(num_tokens * 0.9): ]

train_data.get_device()

0

In [ ]:

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y


xb, yb = get_batch("train")

In [ ]:
print(xb.shape)
print(yb.shape)


torch.Size([8, 32])
torch.Size([8, 32])


In [ ]:

model = BigramLanguageModel(vocab_size)
m = model.to(device)


In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [ ]:


# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.321262 M parameters


In [ ]:

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 8.2773, val loss 8.2791
step 500: train loss 4.0666, val loss 4.7721
step 1000: train loss 3.5686, val loss 4.7253
step 1500: train loss 3.3250, val loss 4.7678
step 2000: train loss 3.1547, val loss 4.7102
step 2500: train loss 3.0557, val loss 4.7872
step 3000: train loss 2.9688, val loss 4.7575
step 3500: train loss 2.8325, val loss 4.7743
step 4000: train loss 2.8177, val loss 4.7688
step 4500: train loss 2.7339, val loss 4.8633
step 5000: train loss 2.6566, val loss 4.8816
step 5500: train loss 2.6221, val loss 4.8558
step 6000: train loss 2.5902, val loss 4.8309
step 6500: train loss 2.5403, val loss 4.9094
step 7000: train loss 2.4820, val loss 4.9122
step 7500: train loss 2.4661, val loss 4.9824
step 8000: train loss 2.4457, val loss 4.9802
step 8500: train loss 2.4022, val loss 5.0467
step 9000: train loss 2.3610, val loss 5.0611
step 9500: train loss 2.3686, val loss 5.0672
step 9999: train loss 2.3364, val loss 5.0665


In [ ]:
torch.save(model.state_dict(), "wordModel")

In [ ]:
xb, yb = get_batch('val')
_idx = model.generate(xb, 100)

print(_idx.shape)

torch.Size([8, 132])


In [ ]:

for batch in _idx:
    res = []
    for num in batch:
        num2 = deordinalize(int(num))
        res.append(num2)
    resstr = tokenizer.decode(res)
    print(resstr)


 between two burdens:
And he saw that rest was good, and the land that it was
pleasant; and bowed his shoulder to bear, and became a
Benjamin's servants.
And he lodged the LORD had about.
And they took God divided the LORD God's hand of a fountain with his
flood.
years also he fed in the earth wrestled and their faces
afterward, and to all the field; to be in the house of the children of the
 Bela was lift up his firstall the daughters of the Egypt; there will not they
because they sent this manner of the cattle from thence into the
 things, that one told Joseph,
Behold, thy father is si and he took with him his two sons,
Manasseh and Ephra for a son: the dream, which
wm, and looked after his sons, and they lift up, and, and
ye pleased Joseph out of the seed house in the droves,
cah, the Syrian, reigned in the the vale of the time of the wombs of had;
And Abraham's son Abraham dwelt his name Beno: but Pharaoh.
Isra butlership again not a little one to be called
which the power with him

In [ ]:

# xb, yb = get_batch('train')
# _idx = model.generate(xb, 100)

# print(_idx.shape)

# for batch in _idx:
#     res = []
#     for num in batch:
#         num2 = deordinalize(int(num))
#         res.append(num2)
#     resstr = tokenizer.decode(res)
#     print(resstr)
